In [19]:
from PIL import Image
from getpass import getpass
from Crypto.Cipher import AES
from Crypto.Protocol.KDF import PBKDF2
import os
import struct

cover_file = "/Users/raviranjan/Documents/cover_file.png"
embed_file = "/Users/raviranjan/Documents/secret_essay.txt"
stego_file = "/Users/raviranjan/Documents/Steganography/art.png"

# ---- Step 1: Read cover image ----
img = Image.open(cover_file)
if img.mode != "RGB":
    img = img.convert("RGB")
pixels = list(img.getdata())

# ---- Step 2: Read & encrypt file ----
data = open(embed_file, "rb").read()
password = input("Enter passphrase: ")
passphrase = password
salt = os.urandom(16)
key = PBKDF2(passphrase, salt, dkLen=32, count=100000)
cipher = AES.new(key, AES.MODE_GCM)
ciphertext, tag = cipher.encrypt_and_digest(data)

# format: [salt][nonce][tag][length][ciphertext]
payload = salt + cipher.nonce + tag + struct.pack(">I", len(ciphertext)) + ciphertext
bits = ''.join(format(byte, '08b') for byte in payload)

# ---- Step 3: Hide bits in LSB of pixels ----
if len(bits) > len(pixels) * 3:
    raise ValueError("Message too large for this image.")

new_pixels = []
bit_idx = 0
for r, g, b in pixels:
    if bit_idx < len(bits):
        r = (r & ~1) | int(bits[bit_idx]); bit_idx += 1
    if bit_idx < len(bits):
        g = (g & ~1) | int(bits[bit_idx]); bit_idx += 1
    if bit_idx < len(bits):
        b = (b & ~1) | int(bits[bit_idx]); bit_idx += 1
    new_pixels.append((r, g, b))

img.putdata(new_pixels)
img.save(stego_file, "PNG")
print(f"✅ Hidden data embedded and saved as {stego_file}")


Enter passphrase: art
✅ Hidden data embedded and saved as /Users/raviranjan/Documents/Steganography/art.png


In [20]:
from PIL import Image
from getpass import getpass
from Crypto.Cipher import AES
from Crypto.Protocol.KDF import PBKDF2
import struct
print("Libraries Imported...")

stego_file = "/Users/raviranjan/Documents/Steganography/art.png"
output_file = "/Users/raviranjan/Documents/Steganography/decoded_file.txt"
print("Steganography and Output file set...")

img = Image.open(stego_file)
pixels = list(img.getdata())
print("Image pixel data extraction done...")

print("Converting RGB Pixels into bits (takes time)...")
bits = ""
for r, g, b in pixels:
    bits += str(r & 1)
    bits += str(g & 1)
    bits += str(b & 1)
print("RGB Pixels conversion complete...")

# Convert bits to bytes
data = int(bits, 2).to_bytes((len(bits) + 7) // 8, byteorder="big")
print("Converted bits into bytes...")

# Parse header
salt = data[:16]
nonce = data[16:32]
tag = data[32:48]
length = struct.unpack(">I", data[48:52])[0]
ciphertext = data[52:52+length]
print("Parse header done...")
print()

password = input("Enter passphrase: ")
passphrase = password
key = PBKDF2(passphrase, salt, dkLen=32, count=100000)
cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
plaintext = cipher.decrypt_and_verify(ciphertext, tag)
print("Decryption Successful!")
print()

with open(output_file, "wb") as f:
    f.write(plaintext)
print(f"✅ Extracted and saved to {output_file}")


Libraries Imported...
Steganography and Output file set...
Image pixel data extraction done...
Converting RGB Pixels into bits...
RGB Pixels conversion complete...
Converted bits into bytes...
Parse header done...

Enter passphrase: art
Decryption Successful!

✅ Extracted and saved to /Users/raviranjan/Documents/Steganography/decoded_file.txt
